<h1 style="text-align: center;"> Hamming Code Simulation Project (Jupyter Notebook) </h1>

- Author: Aiden Rader
- Date: [07/23/2025]  
- Course: EECS 3150 - Summer 2025 - Data Communications  
- Professor: Dr. Junghwan Kim  


---

### Project Description:
Design a Binary Hamming Code of (n, k) = (15, 11) using generator polynomial g(x) = 1 + x + x<sup>4</sup>.

#### Steps:
1. Form a Parity check matrix H (Systematic form)
2. Form a corresponding <b>Generator matrix G</b> (Systematic form)
3. Form a <b>Syndrome Table</b> with corresponding for <u>all Error Patterns for Single error and
Double errors.</u>
4. Encode the following message(s) vectors and give respective Encoded Codeword V:
   1. U<sub>1</sub> = (1 1 1 1 0 0 0 0 0 1 1)
   2. U<sub>2</sub> = (0 1 0 0 1 0 1 0 1 0 0)
   3. U<sub>3</sub> = (1 1 0 1 1 0 0 0 1 1 0)
   4. U<sub>4</sub> = (1 1 1 1 1 0 0 1 1 1 0)
   5. U<sub>5</sub> = (1 0 1 0 1 0 1 0 1 1 1)
5. Decode the following received vectors (find out respective Message vector U):
   1. R<sub>1</sub> = (1 1 1 1 0 0 0 1 0 1 0 1 1 0 1)
   2. R<sub>2</sub> = (1 0 1 0 1 0 0 1 0 1 0 0 1 1 0)
   3. R<sub>3</sub> = (0 0 0 1 1 0 1 0 0 1 1 1 0 1 1)
   4. R<sub>4</sub> = (1 0 1 0 1 0 1 1 0 1 0 1 0 1 1)
   5. R<sub>5</sub> = (1 1 1 1 0 0 0 0 0 0 1 1 0 1 0)

#### Notes:
- In case, you cannot decode any of the given Rxed codewords based on Syndrome
Table, give sufficient reasoning for each case.

---

Import necessary libraries

In [160]:
import numpy as np  # Used for matrix operations
from pprint import pprint # Used for pretty-printing the syndrome table (if needed of course)

# Confirmation message
print("Libraries Imported!")

Libraries Imported!


---

Define generator polynomial in binary 

<i> (this is for debugging reasons and will not be applied when user enters the polynomial) </i>

In [161]:
n, k = 15, 11  # n is the codeword length, k is the message length
r = n - k

test_gen_poly = [1, 1, 0, 0, 1]  # Generator polynomial derived from (1 * x^0 + 1 * x^1 + 0 * x^2 + 0 * x^3 + 1 * x^4)
len_test_gen_poly = len(test_gen_poly)

print("n = ", n)
print("k = ", k)
print("r = ", r)
print("Generator Polynomial: ", test_gen_poly, "\nLength of Generator Polynomial: ", len_test_gen_poly)

n =  15
k =  11
r =  4
Generator Polynomial:  [1, 1, 0, 0, 1] 
Length of Generator Polynomial:  5


---

Create a Mod 2 Division function that can be used to perform the division for the generator matrix

In [162]:
# Helper Function to perform the modulo 2 division, used to return the remainder of the division
def mod2_divide(dividend, divisor):
	dividend = dividend.copy()

	for i in range(len(dividend) - len(divisor) + 1):
		if dividend[i] == 1:
			dividend[i:i+len(divisor)] ^= divisor

	# Return the remainder of the division
	return dividend[-(len(divisor)-1):]

print("Modulo 2 Division Function Defined...")


Modulo 2 Division Function Defined...


---

Now create a function to build the generator matrix using systematic form [I | P]

In [163]:
def build_generator_matrix(gen_poly):
	r = n - k
	G = np.zeros((k, n), dtype=int)

	# Fill identity part
	G[:, :k] = np.eye(k, dtype=int)

	# Fill parity part
	for i in range(k):
		dividend = np.zeros(k + r, dtype=int)
		dividend[i + r] = 1  # x^(r+i)

		# Perform mod-2 division (using mod2 helper function) by g(x)
		remainder = mod2_divide(dividend, gen_poly)

		# Put remainder into parity part of G
		G[i, k:] = remainder
	return G

print("Generator Matrix Function Defined...")

Generator Matrix Function Defined...


Print out the result!

In [164]:
G = build_generator_matrix(test_gen_poly)
print("G matrix:\n\n", G)

G matrix:

 [[1 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]]


---

Now that we have a generator matrix, we can create a function to build the parity check matrix

In [165]:
def build_parity_check_matrix(G):
	r = n - k
	P = G[:, k:]

	# H = [P^T | I_r]
	H = np.hstack((
			P.T,  # Transpose of parity portion of G
			np.eye(r, dtype=int)  # The 4x4 identity matrix
		))

	return H

print("Parity Check Matrix Function Defined...")

Parity Check Matrix Function Defined...


<i> NOTE: Using hstack from numpy to build the parity check matrix is used to horizontally stack the two matrices. </i>

Try and see if building a parity check matrix works using the genorator matrix

In [166]:
H = build_parity_check_matrix(G)
print("H matrix:\n\n", H)

H matrix:

 [[1 0 1 0 1 1 1 1 0 0 0 1 0 0 0]
 [0 1 1 1 1 0 0 0 1 0 0 0 1 0 0]
 [1 0 1 1 1 1 0 0 0 1 0 0 0 1 0]
 [0 1 0 1 1 1 1 0 0 0 1 0 0 0 1]]


---

Now that we have a parity check matrix, we can build a syndrome table out of the parity check matrix using the equation s = H * e^T

In [167]:
# Build the syndrome table
def build_syndrome_table(H):
	n = H.shape[1]
	syndrome_table = {}

	# Check 1-bit errors
	for i in range(n):
		error = np.zeros(n, dtype=int)
		error[i] = 1
		syndrome = (H @ error.T) % 2  # Compute syndrome using matrix multiplication
		syndrome_str = ''.join(map(str, syndrome))

		# Store if not already seen
		if syndrome_str not in syndrome_table:
			syndrome_table[syndrome_str] = error

	# Check 2-bit errors
	for i in range(n):
		for j in range(i+1, n):
			error = np.zeros(n, dtype=int)
			error[i] = 1
			error[j] = 1
			syndrome = (H @ error.T) % 2
			syndrome_str = ''.join(map(str, syndrome))

			if syndrome_str not in syndrome_table:
				syndrome_table[syndrome_str] = error

	return syndrome_table

print("Build Syndrome Table Function Defined...")

Build Syndrome Table Function Defined...


In [168]:
# Helper function to print the syndrome table in a readable format
def print_syndrome_table(syndrome_table):
	print("Syndrome Table (Syndrome -> Error Vector):\n")
	sorted_syn_table = sorted(syndrome_table.items())

	# Print each syndrome and its corresponding error vector
	for syndrome, error in sorted(sorted_syn_table):
		error_str = ' '.join(map(str, error))
		print(f"Syndrome: {syndrome} -> Error Vector: [{error_str}]")

Now build a syndrome table using the parity check matrix

In [169]:
s = build_syndrome_table(H)
print_syndrome_table(s)

Syndrome Table (Syndrome -> Error Vector):

Syndrome: 0000 -> Error Vector: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
Syndrome: 0001 -> Error Vector: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Syndrome: 0010 -> Error Vector: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Syndrome: 0011 -> Error Vector: [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
Syndrome: 0100 -> Error Vector: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Syndrome: 0101 -> Error Vector: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Syndrome: 0110 -> Error Vector: [0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
Syndrome: 0111 -> Error Vector: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
Syndrome: 1000 -> Error Vector: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Syndrome: 1001 -> Error Vector: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
Syndrome: 1010 -> Error Vector: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Syndrome: 1011 -> Error Vector: [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Syndrome: 1100 -> Error Vector: [0 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
Syndrome: 1101 -> Error Vector: [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
Syndrome: 1110 -> Error Vector: [0 0 1 0 0 0 0 0 0 0 0 0 0 0

As we can see here, since our hamming code is (15, 11) our r is 4. This means that the syndrome vector has 4 bits because the parity-check matrix H is size 4 x 15.

- In other examples like in the lecture notes, the commonly used Hamming Code would look something like (7, 4) which means that the syndrome vector has 3 bits. Since this is a bigger code, the syndrome vector is bigger.

---

I wanted to make a class to hold all of our functionality for encoding and decoding. This would be beneficial and lead to ease of use for when we want to enter data ourselves.

The class is as follows:
 - Initialize the class with the generator polynomial
 - Define the encode and decode functions
 - Return all of the function built for encoding and decoding

In [170]:
class LinearCode:
	"""
	A class to represent a linear code for encoding and decoding messages.
	Attributes:
		gen_poly: The generator polynomial used to create the code.
		G: The generator matrix.
		H: The parity check matrix.
		k: The number of message bits.
		n: The total number of bits.
		syndrome_table: A dictionary mapping syndromes to error vectors.
	"""

	# Initialize the class
	def __init__(self, gen_poly, n, k):
		self.gen_poly = gen_poly
		self.n = n
		self.k = k
		self.G = build_generator_matrix(gen_poly)
		self.H = build_parity_check_matrix(self.G)
		self.k, self.n = self.G.shape
		self.syndrome_table = build_syndrome_table(self.H)

	# Call for printing matrices
	def print_info(self):
		print("For Hamming Code with n =", self.n, "and k =", self.k, "and generator polynomial g(x) =", self.gen_poly, ":\n")

		print("\n=== Generator Matrix (G) ===")
		print(self.G)

		print("\n=== Parity Check Matrix (H) ===")
		print(self.H)
		print("\n")

		print("\n=== Syndrome Table ===")
		print_syndrome_table(self.syndrome_table)  # special function to format the syndrome table
		print("\n")

		return

	# Helper function to compute the syndrome
	def compute_syndrome(self, received_vector):
		received_vector = np.array(received_vector, dtype=int)
		return (self.H @ received_vector.T) % 2

	# Encode a message
	def encode(self, message):
		message = np.array(message, dtype=int)
		if len(message) != self.k:
			raise ValueError(f"Message must be of length {self.k}, returned {len(message)}")
		return (message @ self.G) % 2

	# Decode a message
	def decode(self, message):
		message = np.array(message, dtype=int)
		syndrome = self.compute_syndrome(message)
		syndrome_str = ''.join(map(str, syndrome))

		if syndrome_str == '0000':
			# For when there is no error detected
			return message[:self.k], message, syndrome_str, None

		if syndrome_str in self.syndrome_table:
			# Correctable error found
			error_vector = self.syndrome_table[syndrome_str]
			corrected = (message + error_vector) % 2
			return corrected[:self.k], corrected, syndrome_str, error_vector

		# Uncorrectable error
		return None, message, syndrome_str, None


Now that we have everything set up, we can start encoding and decoding! I wanted to make a class system for encoding and decoding and then transfering all of these functions into it! So here it is.

In [171]:
def encode_message(code, messages):
	# Ensure input is a list of messages
	if isinstance(messages, np.ndarray):
		messages = [messages]

	len_messages = len(messages)

	print("Encoding ", len_messages, " Messages... \n")

	for i, m in enumerate(messages, 1):
		V = code.encode(m)
		print(f"U{i}: {m}")
		print(f"V{i} (Encoded): {V}\n")

In [ ]:
def decode_message(code, messages):
	# Ensure input is a list of messages
	if isinstance(messages, np.ndarray):
		messages = [messages]

	len_messages = len(messages)

	# Debug statement
	print("Decoding ", len_messages, " Messages... \n")

	for i, r in enumerate(messages, 1):
		message, syndrome, error = code.decode(r)
		print(f"R{i}: {r}")
		print(f"Syndrome for R{i}: {syndrome}")

		# Print the decoded message or error information
		if syndrome == '0000':
			print(f"V{i} (Decoded): {message}\n")
		elif error is not None:
			print(f"V{i} (Decoded with correction): {message}\n(Corrected from {r} with error {error})\n")
		else:
			print(f"V{i} (Uncorrectable error): {r} (Syndrome: {syndrome})\n")

---

Now we can encode and decode with our values given in the problem statement! This will be where we get our answers!

In [177]:
gen_poly = [1, 1, 0, 0, 1]  # Our generator polynomial
n, k = 15, 11  # Codeword length and message length
hamming_code = LinearCode(gen_poly, n, k)  # Create an instance of LinearCode

print("Hamming Code Simulation Project Initialized\n")

hamming_code.print_info()  # Print the matrices

encode_messages = [
	np.array([1,1,1,1,0,0,0,0,0,1,1]),  # U1
	np.array([0,1,0,0,1,0,1,0,1,0,0]),  # U2
	np.array([1,1,0,1,1,0,0,0,1,1,0]),  # U3
	np.array([1,1,1,1,1,0,0,1,1,1,0]),  # U4
	np.array([1,0,1,0,1,0,1,0,1,1,1])   # U5
]

decode_messages = [
	np.array([1,1,1,1,0,0,0,1,0,1,0,1,1,0,1]),  # R1
	np.array([1,0,1,0,1,0,0,1,0,1,0,0,1,1,0]),  # R2
	np.array([0,0,0,1,1,0,1,0,0,1,1,1,0,1,1]),  # R3
	np.array([1,0,1,0,1,0,1,1,0,1,0,1,0,1,1]),  # R4
	np.array([1,1,1,1,0,0,0,0,0,0,1,1,0,1,0])   # R5
]

encode_message(hamming_code, encode_messages)
decode_message(hamming_code, decode_messages)

print("\nSimulation Project Complete!")


Hamming Code Simulation Project Initialized

For Hamming Code with n = 15 and k = 11 and generator polynomial g(x) = [1, 1, 0, 0, 1] :


=== Generator Matrix (G) ===
[[1 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]]

=== Parity Check Matrix (H) ===
[[1 0 1 0 1 1 1 1 0 0 0 1 0 0 0]
 [0 1 1 1 1 0 0 0 1 0 0 0 1 0 0]
 [1 0 1 1 1 1 0 0 0 1 0 0 0 1 0]
 [0 1 0 1 1 1 1 0 0 0 1 0 0 0 1]]



=== Syndrome Table ===
Syndrome Table (Syndrome -> Error Vector):

Syndrome: 0000 -> Error Vector: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
Syndrome: 0001 -> Error Vector: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Syndrome: 0010 -> Error Vector: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Syndrome: 0011 -> Error Vector: [1 0 0 0 0